# Prepare

In [106]:
# machine learning modules
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures

from sklearn.neighbors import KNeighborsRegressor

In [49]:
# general modules
import pandas as pd
import sqlite3 
import os
import numpy as np
import pandas_profiling
import seaborn as sns
import sqlite3 as sql
import hashlib

import pickle

In [22]:
# variables
randon_state = 42

temp_path = 'temp'
pckl_train = 'train_data'
pckl_test = 'test_data'

# Import

In [23]:
train_in = open(os.path.join("..",temp_path, pckl_train),'rb')
df_train = pickle.load(train_in)
train_in.close()

In [24]:
test_in = open(os.path.join("..",temp_path, pckl_test),'rb')
df_test = pickle.load(test_in)
test_in.close()

In [25]:
df_train.columns

Index(['order_date', 'ship_date', 'ship_mode', 'customer_id', 'customer_name',
       'segment', 'city', 'state', 'country', 'postal_code', 'market',
       'region', 'product_id', 'category', 'sub-category', 'product_name',
       'sales', 'quantity', 'discount', 'profit', 'shipping_cost',
       'order_priority', 'returned', 'ship_delay'],
      dtype='object')

In [26]:
df_train

,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,country,postal_code,...,sub-category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,returned,ship_delay
index,,,,,,,,,,,,,,,,,,,,,
27507,2012-12-27,2012-12-31,Standard Class,SP-20860,a7d03c30d416fc5f7d695b495884fdd7,Corporate,Murfreesboro,Tennessee,United States,37130.0,...,Accessories,SanDisk Ultra 16 GB MicroSDHC Class 10 Memory ...,72.74400,7,0.200,-12.73020,6.720,Medium,No,4
35511,2014-12-25,2015-01-01,Standard Class,JD-15895,1b2850c124acd1bc24237b4b5228b65e,Corporate,Oosterhout,North Brabant,Netherlands,NaN,...,Labels,"Smead File Folder Labels, Adjustable",23.73000,7,0.500,-21.00000,3.430,Low,No,7
9172,2012-05-08,2012-05-11,Second Class,AB-10600,6acab08bb2b385c8569adfd24730ee01,Corporate,Phnom Penh,Phnom Penh,Cambodia,NaN,...,Bookcases,"Dania Corner Shelving, Pine",617.10000,5,0.000,172.65000,36.380,Medium,No,3
31366,2011-06-30,2011-07-02,Second Class,GH-14410,1528a0a296f3ecf500753855ea9a21a5,Home Office,Lima,Lima (city),Peru,NaN,...,Chairs,"Hon Bag Chairs, Red",54.18000,3,0.400,-32.52000,4.919,Medium,Yes,2
24465,2013-06-23,2013-06-26,First Class,KW-16435,648a7c6f93ee0f453ee1378466a84ff8,Consumer,London,England,United Kingdom,NaN,...,Storage,"Fellowes Box, Wire Frame",50.62500,3,0.100,20.20500,8.570,Medium,No,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,2014-07-10,2014-07-14,Second Class,PS-18970,c90d076ff45727789cb1742f443028e1,Home Office,Petapa,Guatemala,Guatemala,NaN,...,Bookcases,"Bush Corner Shelving, Metal",246.90000,3,0.000,32.04000,28.644,Medium,No,4
44732,2014-11-26,2014-12-02,Standard Class,CK-12205,8fe3138a7ef91d7f8635f63b9d5331ad,Consumer,Panama City,Panama,Panama,NaN,...,Labels,"Hon Round Labels, Alphabetical",15.55200,6,0.400,1.99200,1.281,Medium,No,6
38158,2011-10-14,2011-10-18,Second Class,LR-17035,a916b8bb7b9fcce602d0808e2eef7979,Corporate,Agra,Uttar Pradesh,India,NaN,...,Labels,"Hon Shipping Labels, Alphabetical",44.76000,4,0.000,20.04000,2.690,High,No,4


# Transform

In [20]:
target_col = "ship_delay"

In [68]:
date_cols = ["order_date"]
num_cols = ["sales", "quantity", "profit", "shipping_cost"]
std_cols = ["discount"]

nomi_cols = ["segment", "market", "category", ]
ordi_cols = ["ship_mode", "order_priority"]


bin_cols = ["returned"]

unused_cols = ["ship_date", "customer_id", "customer_name", "city", "state" ,"country", "postal_code", "region", "product_id",
               "sub-category", "product_name"]



In [70]:
# check column count

used_cols_len = len(date_cols)+len(num_cols)+len(nomi_cols)+len(ordi_cols)+len(tf_cols)+len(unused_cols)+len(std_cols)+1

print("Total cols:", len(df_train.columns))
print("Considered cols:", used_cols_len)

Total cols: 24
Considered cols: 24


In [71]:
def split_x_y(df):
    y = np.array(df[target_col])
    return df, y

In [129]:
X_train, y_train = split_x_y(df_train)

In [130]:
X_test, y_test = split_x_y(df_test)

### Pipeline

In [169]:
ordinal_categories = [
    ["Standard Class", "Second Class", "First Class", "Same Day"],
    ["Low", "Medium", "High", "Critical"]
]

In [170]:
ordinal_encoder = OrdinalEncoder(categories=ordinal_categories)

In [171]:
std_pipeline = Pipeline([
    ("simple_imp", SimpleImputer())
])

In [172]:
num_pipeline = Pipeline([
    ("simple_imp", SimpleImputer()),
    ("pol", PolynomialFeatures()),
])

In [173]:
nomi_pipeline = Pipeline([
    ("one_hot", OneHotEncoder()),
])

In [174]:
transform_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_cols),
    ("nomi", nomi_pipeline, nomi_cols),
    #("ordi", ordinal_encoder, ordi_cols),
    ("std", std_pipeline, std_cols)
])

### KNearest

In [175]:
regression_pipeline = Pipeline([
    ("transform", transform_pipeline),
    ("knearest", KNeighborsRegressor(n_neighbors=1)),
])


In [176]:
model = regression_pipeline.fit(X_train, y_train)


In [177]:
model.score(X_test, y_test)

0.9998363834361753

In [178]:
y_test_predict = model.predict(X_test)

In [179]:
pd.DataFrame([y_test_predict]).transpose()

,0
0,4.0
1,7.0
2,3.0
3,2.0
4,3.0
...,...
38466,4.0
38467,6.0
38468,4.0
38469,2.0


In [180]:
df_check = X_test.copy() 

In [181]:
df_check["prediction"] = y_test_predict

In [182]:
df_check.loc[df_check["ship_delay"]!=df_check["prediction"]]

,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,country,postal_code,...,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,returned,ship_delay,prediction
index,,,,,,,,,,,,,,,,,,,,,
45837,2012-12-31,2013-01-05,Standard Class,DB-2970,8eaed2ccbc41dbe05befdddfefca57bd,Corporate,Sanandaj,Kordestan,Iran,NaN,...,"Sanford Pens, Blue",22.200,2,0.0,5.2800,1.09,Medium,0,5,4.0
43660,2013-08-10,2013-08-14,Standard Class,CA-12775,4bda0c23717215e463478306ee6d3421,Consumer,Hamm,North Rhine-Westphalia,Germany,NaN,...,"Wilson Jones Hole Reinforcements, Clear",7.980,2,0.0,0.8400,1.48,High,0,4,3.0
44307,2014-10-29,2014-11-02,Second Class,KH-16510,e0d3c49243a23c0f6e28f9164b7c1e38,Consumer,Houston,Texas,United States,77095.0,...,Xerox 21,15.552,3,0.2,5.4432,1.36,High,0,4,5.0
42532,2012-07-02,2012-07-09,Standard Class,KB-16315,95afff0a53d0eca5df41985f7500bebe,Consumer,Hollywood,Florida,United States,33021.0,...,Xerox 1957,15.552,3,0.2,5.6376,1.71,Medium,0,7,3.0


In [150]:
type(result_test)

list

In [151]:
result_test

[array([4., 7., 3., ..., 4., 2., 0.]),
 array([4, 7, 3, ..., 4, 2, 0], dtype=int64)]

In [ ]:
model = KNeighborsRegressor(n_neighbors=4).fit(X_train_num, y_train)
y_train_predict = model.predict(X_train_num)
